In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import streamlit as st

In [3]:
##Step 1: Read CSV File
df = pd.read_csv("/Users/quantum/Desktop/recomender/input/movie_dataset.csv")
df.head(3)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes


In [33]:
df['title'] =df['title'].apply(lambda x: x.lower())

In [35]:
df.head(5)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director,Com_features
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron,culture clash future space war space colony so...
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",pirates of the caribbean: at world's end,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski,ocean drug abuse exotic island east india trad...
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes,spy based on novel secret agent sequel mi6 Dan...
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,the dark knight rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan,dc comics crime fighter terrorist secret ident...
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",john carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton,based on novel mars medallion space travel pri...


In [4]:
##Step 2: Select Features
com_features = ['keywords','cast','genres','director']

In [5]:
##Step 3: Create a column in DF which combines all selected features
for f in com_features:
    df[f] = df[f].fillna('')
def com_features(row):
    try:
        return row['keywords'] +" "+row['cast']+" "+row['genres']+" "+row['director']
    except:
        print("Error:",row)
df['Com_features'] = df.apply(com_features,axis=1)
df['Com_features'].head()

0    culture clash future space war space colony so...
1    ocean drug abuse exotic island east india trad...
2    spy based on novel secret agent sequel mi6 Dan...
3    dc comics crime fighter terrorist secret ident...
4    based on novel mars medallion space travel pri...
Name: Com_features, dtype: object

In [6]:
##Step 4: Create count matrix from this new combined column
cv = CountVectorizer()
count_matrix = cv.fit_transform(df['Com_features'])

In [7]:
##Step 5: Compute the Cosine Similarity based on the count_matrix
cosine_similarity = cosine_similarity(count_matrix)


In [36]:
#Input from the user
def movie_input():
    movie = input('Enter the movie name: ')
    return movie.lower()

In [49]:
## Step 6: Get index of this movie from its title
def get_index_from_title(title):
    return df[df.title == title]["index"].values[0]
movie_inp =st.in
#Movie index from the data
movie_index =get_index_from_title(movie_input())

#Enumerating with movie index like (1,0.4),(2,0.6),(3,0.9) etc
similar_movies = list(enumerate(cosine_similarity[movie_index]))

Enter the movie name: star trek beyond


In [50]:
## Step 7: Get a list of similar movies in descending order of similarity score

sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)

In [52]:
## Printing top 10 movies similar to the given index

def get_title_from_index(index):
	return df[df.index == index]["title"].values[0]

for movie in sorted_similar_movies[1:50]:
    print(get_title_from_index(movie[0]))

star trek into darkness
star trek
guardians of the galaxy
avatar
the helix... loaded
avengers: age of ultron
star wars: clone wars: volume 1
the chronicles of riddick
the time machine
six-string samurai
captain america: civil war
star trek: insurrection
moonraker
wing commander
planet of the apes
paul
mission: impossible - rogue nation
mission: impossible - ghost protocol
dredd
mad max beyond thunderdome
the avengers
captain america: the winter soldier
riddick
oblivion
star wars: episode iii - revenge of the sith
the fifth element
titan a.e.
the world's end
u.f.o.
jurassic world
transformers: age of extinction
jupiter ascending
red planet
carriers
avp: alien vs. predator
dragonball evolution
the empire strikes back
john carter
superman returns
the black hole
silent running
captain america: the first avenger
starship troopers
zathura: a space adventure
priest
robocop 3
walking with dinosaurs
men in black
street fighter: the legend of chun-li
